In [17]:
import pandas as pd
from sqlalchemy import create_engine, text

engine = create_engine(
    f"postgresql+psycopg2://postgres:1510261105@"
    f"localhost:5432/neiro-insight"
)

with engine.connect() as conn:
    result = conn.execute(text("SELECT * from audio_dialogs LIMIT 1")).fetchone()
    res = dict(result._asdict()) if result else None
res

pd.set_option('display.max_columns', 500)

In [256]:
df = pd.read_sql(f"""
SELECT *
    -- audio_dialog_fk_id
    -- max(t1.has_greeting::int) has_greeting,
    -- max(t1.has_name::int) has_name,
    -- array_agg(parasite_words) parasite_words,
    -- max(t1.has_stopwords::int) has_stopwords,
    -- max(t1.has_swear_words::int) has_swear_words
-- t2.audio_dialog_fk_id, t1.*, speaker_id
from dialog_criterias t1
join dialog_rows t2 on t1.dialog_row_fk_id = t2.id
left join audio_dialogs t3 on t2.audio_dialog_fk_id = t3.id
-- left join audio_dialog t3 on t2.audio_dialog_fk_id = t3.id
where 1=1
and t3.updated_at >= '2025-08-03'
-- speaker_id = 'SALES'
-- and swear_words not in ('""', '')
-- and swear_words is not null
""", engine)
df

,dialog_criteria_id,dialog_row_fk_id,greeting_phrase,found_name,farewell_phrase,interjections,parasite_words,abbreviations,slang,inappropriate_phrases,diminutives,stop_words,swear_words,non_professional_phrases,order_offer,order_processing,order_resume,await_requests,id,audio_dialog_fk_id,row_num,row_text,speaker_id,start,end,id,file_name,status,duration,processing_time,created_at,updated_at
0,335b9ce4-86f1-478d-bc7d-6239ced9248a,0aea6868-de93-4847-8461-daf20d4f3106,,,,"""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""",None,0aea6868-de93-4847-8461-daf20d4f3106,96a51959-427a-44ce-ad08-157ac53d8251,85,что он отходит на столе. Буквально минуту офор...,SALES,688.1,716.9,96a51959-427a-44ce-ad08-157ac53d8251,in-07010101-89687474657-20250416-101305-174478...,PROCESSED,817.54,195.245297,2025-06-26 01:28:27,2025-08-05 02:10:01.661396
1,130be468-6c01-4315-a13f-63327d2b10f8,0e94bb5e-7dde-4982-a2dd-576520c0f1c1,,,,"""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""",None,0e94bb5e-7dde-4982-a2dd-576520c0f1c1,96a51959-427a-44ce-ad08-157ac53d8251,87,Сонар автомобиля подскажите.,SALES,729.9,732.1,96a51959-427a-44ce-ad08-157ac53d8251,in-07010101-89687474657-20250416-101305-174478...,PROCESSED,817.54,195.245297,2025-06-26 01:28:27,2025-08-05 02:10:01.661396
2,9d92adf2-1717-4512-841e-e0a4961c33be,286effbd-ffe1-45e2-b3a1-1462ebae80c8,,Ольга,,"""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""",None,286effbd-ffe1-45e2-b3a1-1462ebae80c8,96a51959-427a-44ce-ad08-157ac53d8251,91,"Давайте проверим. О, Ольга, 455.",SALES,774.0,777.7,96a51959-427a-44ce-ad08-157ac53d8251,in-07010101-89687474657-20250416-101305-174478...,PROCESSED,817.54,195.245297,2025-06-26 01:28:27,2025-08-05 02:10:01.661396
3,dc74d1e5-91e8-490d-8960-aa56dbbf28d2,469490bd-8d30-4b76-98c7-183bfdc9b884,,,,"""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""",None,469490bd-8d30-4b76-98c7-183bfdc9b884,96a51959-427a-44ce-ad08-157ac53d8251,93,"Указываю, что у вас наш подарок.",SALES,782.9,788.2,96a51959-427a-44ce-ad08-157ac53d8251,in-07010101-89687474657-20250416-101305-174478...,PROCESSED,817.54,195.245297,2025-06-26 01:28:27,2025-08-05 02:10:01.661396
4,155e143a-6807-4ebf-a5fc-bf08f6deaeab,d1374ace-db24-45e9-8d3a-40759bc12316,,салават,,"""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""",None,d1374ace-db24-45e9-8d3a-40759bc12316,96a51959-427a-44ce-ad08-157ac53d8251,95,"Дальше салават. Дорожный первый проезд М5А, с...",SALES,792.4,794.8,96a51959-427a-44ce-ad08-157ac53d8251,in-07010101-89687474657-20250416-101305-174478...,PROCESSED,817.54,195.245297,2025-06-26 01:28:27,2025-08-05 02:10:01.661396
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130570,a337d222-9b9e-4964-af35-653cbc521f97,c53731f6-672e-4dd5-884a-93d98040a54b,,,,"""""","""то есть""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""",None,c53731f6-672e-4dd5-884a-93d98040a54b,29023750-ea98-4a93-9da6-f238d368325c,31,в москве нет они не хранятся в москве они могу...,SALES,454.7,460.9,29023750-ea98-4a93-9da6-f238d368325c,in-07010101-89142449224-20250408-085934-174409...,PROCESSED,561.78,114.564781,2025-06-25 23:23:44,2025-08-04 19:45:36.879820
130571,9eb63a6e-03d3-406d-b241-8c823497ce2d,01eea76c-d556-4237-bd50-5fa7914f14e5,,,благодарю за обращение,"""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""",None,01eea76c-d556-4237-bd50-5fa7914f14e5,29023750-ea98-4a93-9da6-f238d368325c,34,У нас нас кладет. Благодарю вас за обращение....,SALES,487.7,489.5,29023750-ea98-4a93-9da6-f238d368325c,in-07010101-89142449224-20250408-085934-174409...,PROCESSED,561.78,114.564781,2025-06-25 23:23:44,2025-08-04 19:45:36.879820
130572,c2178d94-499a-4ddb-89fd-f82b54de4ddd,6620dcd6-ba68-4e8e-b9fb-dd4545f58116,,,,"""""","""""","""""","""""","""""","""""","""нет возможности""",

In [260]:
df_test = df[['row_text', 'audio_dialog_fk_id']]
# df_test[df_test['row_text'].str.contains('здравствуйте|добрый день', regex=True)]
df_test['test'] = df_test['row_text'].str.contains('здравствуйте|добрый день', regex=True)
df_test

/var/folders/5p/p26rjdgs7_3_q29p10183wch0000gn/T/ipykernel_85816/4062429564.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['test'] = df_test['row_text'].str.contains('здравствуйте|добрый день', regex=True)


,row_text,audio_dialog_fk_id,test
0,что он отходит на столе. Буквально минуту офор...,96a51959-427a-44ce-ad08-157ac53d8251,False
1,Сонар автомобиля подскажите.,96a51959-427a-44ce-ad08-157ac53d8251,False
2,"Давайте проверим. О, Ольга, 455.",96a51959-427a-44ce-ad08-157ac53d8251,False
3,"Указываю, что у вас наш подарок.",96a51959-427a-44ce-ad08-157ac53d8251,False
4,"Дальше салават. Дорожный первый проезд М5А, с...",96a51959-427a-44ce-ad08-157ac53d8251,False
...,...,...,...
130570,в москве нет они не хранятся в москве они могу...,29023750-ea98-4a93-9da6-f238d368325c,False
130571,У нас нас кладет. Благодарю вас за обращение....,29023750-ea98-4a93-9da6-f238d368325c,False
130572,Поэтому нет возможности уточнить именно по дан...,29023750-ea98-4a93-9da6-f238d368325c,False
130573,"Да, надеюсь, после того, как они...",29023750-ea98-4a93-9da6-f238d368325c,False


In [270]:
import pandas as pd
from fuzzywuzzy import fuzz

# Функция нормализации (оставляем как есть)
def normalize_text(text):
    """Нормализация текста для обработки с учетом возможных ошибок"""
    text = text.lower()
    replacements = {
        'здрасте': 'здравствуйте',
        'здрасьте': 'здравствуйте',
        'здрась': 'здравствуйте',
        'драсьте': 'здравствуйте',
        'доброго дня': 'добрый день',
        'добраго вечера': 'добрый вечер',
        'всего хорошего': 'всего доброго',
        'в общем-то': 'в общем',
        'короче говоря': 'короче',
        'типа того': 'типа',
        'как-бы': 'как бы',
        'т.е.': 'то есть',
        'т.д.': 'так далее',
        'т.п.': 'тому подобное'
    }
    for wrong, correct in replacements.items():
        text = text.replace(wrong, correct)
    return text

# Векторизованная версия функции поиска фраз
def find_phrases_in_df(df, text_column, phrases, threshold=80):
    # Нормализуем весь столбец сразу
    normalized_texts = df[text_column].apply(normalize_text)
    
    # Создаем функцию для поиска по одному тексту
    def find_in_text(text):
        for phrase in phrases:
            if fuzz.partial_ratio(phrase, text) >= threshold:
                return phrase
        return None
    
    # Применяем ко всем текстам
    return normalized_texts.apply(find_in_text)

# Пример использования:
phrases_to_find = ['здравствуйте', 'добрый день', 'всего доброго']
df_test['found_phrase'] = find_phrases_in_df(df_test, 'row_text', phrases_to_find)
import swifter
# df_test['found_phrase'] = df_test['row_text'].swifter.apply(lambda x: find_phrase(x, phrases_to_find))
# Фильтрация строк где найдены фразы
df_test = df_test[df_test['found_phrase'].notna()]
df_test

/var/folders/5p/p26rjdgs7_3_q29p10183wch0000gn/T/ipykernel_85816/405889507.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['found_phrase'] = find_phrases_in_df(df_test, 'row_text', phrases_to_find)


,row_text,audio_dialog_fk_id,test,found_phrase
5,"с давлением, с адресом и временем. А на услови...",96a51959-427a-44ce-ad08-157ac53d8251,False,всего доброго
17,добрый день меня зовут диана как я могу по обр...,86f6ab05-5d97-46e3-928e-c3b4a6c1be0d,True,добрый день
40,Благодарю вас за уделенное время. Всего доброг...,5a352229-06df-4a1b-86df-f308823c0c80,False,всего доброго
50,"Субтитры создавал DimaTorzok Добрый день, мен...",caa19eb1-e035-484c-889e-befa99ab4a70,False,добрый день
68,"Добрый день, меня зовут Кристина. Как я могу к...",1e157a48-ce6d-42f0-9be7-26e38ff9cc59,False,добрый день
...,...,...,...,...
130501,здравствуйте меня зовут наталья как вам могу о...,e822c11c-df9f-4799-8bdc-09a504b5a758,True,здравствуйте
130502,"Хорошо? Перевожу Для Добрый день, меня зовут Э...",e822c11c-df9f-4799-8bdc-09a504b5a758,False,добрый день
130513,"Добрый день, Мистер Вячеслав. обращаться позд...",90572bdd-1711-44b4-9573-a6551fe55bd9,False,добрый день
130525,"Ксения, добрый день, меня Антон зовут. Подска...",90572bdd-1711-44b4-9573-a6551fe55bd9,True,добрый день


In [266]:
df_test['row_text'][5]

'с давлением, с адресом и временем. А на условиях рекомендуем приехать за 10-15 минут заранее. Подскажите, остались ли еще вопросы? Нет, все, спасибо, до свидания.  Благодарю за обращение. Всего доброго. '

In [237]:
df[df['swear_words'] != '""']

,dialog_criteria_id,dialog_row_fk_id,greeting_phrase,found_name,farewell_phrase,interjections,parasite_words,abbreviations,slang,inappropriate_phrases,diminutives,stop_words,swear_words,non_professional_phrases,order_offer,order_processing,order_resume,await_requests,id,audio_dialog_fk_id,row_num,row_text,speaker_id,start,end,id,file_name,status,duration,processing_time,created_at,updated_at
2859,d382c809-8bf2-4397-82ae-708c6f481c63,8db4bb27-59b4-4234-aaae-de420d1ddc58,,,,"""""","""""","""""","""""","""""","""""","""""","""тупица""","""""","""""","""""","""""",None,8db4bb27-59b4-4234-aaae-de420d1ddc58,c3ef19cf-4941-42c0-88e0-d9e81617192a,28,"Тупица 57,1 черный.",SALES,243.2,245.8,c3ef19cf-4941-42c0-88e0-d9e81617192a,in-07010101-89272678188-20250420-104800-174513...,PROCESSED,419.76,141.891079,2025-06-26 18:22:31,2025-08-06 23:28:51.796105
2881,4cde0028-0008-4e04-b207-531db4a62869,f8969c67-5baa-48de-8efd-4406d6a28250,,Ширина,,"""""","""""","""""","""""","""""","""""","""""","""тупица""","""""","""""","""""","""""",None,f8969c67-5baa-48de-8efd-4406d6a28250,ab1599d1-ccff-4f1b-a16d-bf4bed59c99a,15,"15-20. Ширина диска 6, диаметр 15, разболтовка...",SALES,303.9,311.7,ab1599d1-ccff-4f1b-a16d-bf4bed59c99a,in-07010101-89272260085-20250421-164045-174524...,PROCESSED,483.68,102.371904,2025-06-26 18:23:00,2025-08-06 23:30:00.352705
3231,45d89846-a7a0-4c04-891e-da3c0d8acdeb,288cef75-26d4-453f-bea9-248ce63c744d,,,,"""""","""""","""""","""""","""""","""""","""""","""тупица""","""""","""""","""""","""""",None,288cef75-26d4-453f-bea9-248ce63c744d,37737750-4309-4121-b9b9-d09a1dc98ff4,59,"6 на R15, 4 на 114, 3 разболтовка, вылет 46. ...",SALES,501.4,506.4,37737750-4309-4121-b9b9-d09a1dc98ff4,in-07010101-89136830893-20250430-125413-174600...,PROCESSED,660.48,181.661366,2025-06-26 18:32:09,2025-08-06 23:39:29.319977
3704,e360d505-376c-40f5-813c-b424149e8b92,e52a6d17-3a6d-432a-87b8-b3376157cb8f,,,,"""""","""""","""""","""""","""""","""""","""""","""хуя""","""""","""""","""""","""""",None,e52a6d17-3a6d-432a-87b8-b3376157cb8f,fd8ab821-e037-43d1-b6a4-1f86e5718cd1,23,"Какого хуя? Я должен посмотреть все. Может, м...",SALES,219.5,222.8,fd8ab821-e037-43d1-b6a4-1f86e5718cd1,in-07010101-89080498625-20250415-104441-174470...,PROCESSED,502.74,104.988620,2025-06-26 18:47:56,2025-08-06 23:54:04.131937
3910,bd7b8501-5874-438b-bd09-90dd919d9e2c,9e488381-93a3-4859-a5f3-eb646ff72df6,,,,"""""","""ну""","""""","""""","""""","""""","""""","""сука""","""""","""""","""""","""""",None,9e488381-93a3-4859-a5f3-eb646ff72df6,eedd2b24-9e4a-4304-8d60-b49f60e97299,57,"А, ну да, да, сука.",SALES,359.8,361.8,eedd2b24-9e4a-4304-8d60-b49f60e97299,in-07010101-89178760874-20250404-161152-174377...,PROCESSED,729.76,134.734824,2025-06-26 18:51:43,2025-08-06 23:58:10.431950
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124162,8778addd-7420-440c-88ad-ccce5dd89ca9,c449c010-4b27-43e3-86ca-0351ee1fe5b5,,,,"""""","""ну""","""""","""""","""""","""""","""""","""хуя""","""""","""""","""""","""""",None,c449c010-4b27-43e3-86ca-0351ee1fe5b5,d0d75055-f2ab-4073-a230-c391b210441c,12,Ну только меня хуя обожает.,SALES,168.0,170.8,d0d75055-f2ab-4073-a230-c391b210441c,in-07010101-8-20250402-164719-1743601639.21046...,PROCESSED,374.86,182.082497,2025-06-25 23:54:33,2025-08-05 00:34:56.328004
127720,1fd1de15-5aee-410d-ae59-22e70006fbc6,7bf4567b-3082-4eda-9ac2-f567b1bf90b7,,гаджи,,"""""","""""","""""","""""","""""","""""","""""","""гаджи""","""""","""""","""""","""""",None,7bf4567b-3082-4eda-9ac2-f567b1bf90b7,c62ef92b-35ff-40ba-95e2-1f26cd065aee,48,"Нет, примухни миросум, гаджи нам нету.",SALES,277.3,279.4,c62ef92b-35ff-40ba-95e2-1f26cd065aee,in-07010101-89220888398-20250417-154109-174489...,PROCESSED,536.00,220.711125,2025-06-26 02:36:16,2025-08-05 03:16:03.192026
129405,93286874-c3b3-4e1b-aa79-5ead09b93baf,67c2e3d3-901a-43f0-ac32-98af3921178a,,,,"""""","""""","""""","""""","""""","""""","""""","""блядь""","""""",""""""

In [166]:
# df.groupby(['found_name']).agg({'audio_dialog_fk_id': 'nunique'})
# df_name['found_name'].unique()
df_name = df.copy()
df_name = df_name.sort_values(['audio_dialog_fk_id','row_num'])
df_name = df_name.groupby(['audio_dialog_fk_id']).agg({'found_name': 'first'}).reset_index()

df_name.loc[(df_name['found_name'] == '') | (df_name['found_name'].isna()), 'name_flag'] = 0
df_name['name_flag'] = df_name['name_flag'].fillna(1)
# df_name = df_name.groupby(['audio_dialog_fk_id']).agg({'name_flag': 'max'}).reset_index()
df_name = df_name.groupby(['name_flag']).agg({'audio_dialog_fk_id': 'nunique'}).reset_index()
# df_name = df_name[df_name['name_flag'] == 1]
df_name
# tuple(df_name['found_name'].unique())

,name_flag,audio_dialog_fk_id
0,0.0,440
1,1.0,3519


In [181]:
# df.groupby(['found_name']).agg({'audio_dialog_fk_id': 'nunique'})
df_name = df.groupby(['audio_dialog_fk_id']).agg({'greeting_phrase': 'sum'}).reset_index()
# df_name['found_name'].unique()
df_name.loc[(df_name['greeting_phrase'] == '') | (df_name['greeting_phrase'].isna()), 'greeting_phrase_flag'] = 0
df_name['greeting_phrase_flag'] = df_name['greeting_phrase_flag'].fillna(1)
df_name = df_name.groupby(['greeting_phrase_flag']).agg({'audio_dialog_fk_id': 'nunique'}).reset_index()
# df_name[df_name['name_flag'] == 0]
df_name
# df_name['greeting_phrase'].unique()

,greeting_phrase_flag,audio_dialog_fk_id
0,0.0,103
1,1.0,3856


In [180]:
103/3856

0.02671161825726141

In [187]:
# df.groupby(['found_name']).agg({'audio_dialog_fk_id': 'nunique'})
df_farewell = df.groupby(['audio_dialog_fk_id']).agg({'farewell_phrase': 'sum'}).reset_index()
# df_name['found_name'].unique()
df_farewell.loc[(df_farewell['farewell_phrase'] != '') & (~df_farewell['farewell_phrase'].isna()), 'farewell_phrase_flag'] = 1
df_farewell['farewell_phrase_flag'] = df_farewell['farewell_phrase_flag'].fillna(0)
df_farewell = df_farewell.groupby(['farewell_phrase_flag']).agg({'audio_dialog_fk_id': 'nunique'}).reset_index()
# df_farewell[df_farewell['farewell_phrase_flag'] == 0].head(20)
# df_farewell['farewell_phrase'].unique()
df_farewell

,farewell_phrase_flag,audio_dialog_fk_id
0,0.0,1117
1,1.0,2842


In [255]:
# df.groupby(['found_name']).agg({'audio_dialog_fk_id': 'nunique'})
# df_name['found_name'].unique()
df_farewell = df.copy()
df_farewell.loc[df_farewell['stop_words'] == '""', 'stop_words'] = np.nan
df_farewell = df_farewell.groupby(['audio_dialog_fk_id']).agg({'stop_words': 'sum'}).reset_index()

df_farewell.loc[(df_farewell['stop_words'].isin(('', '0'))) | (df_farewell['stop_words'].isna()), 'stop_words_flag'] = 0
df_farewell['stop_words_flag'] = df_farewell['stop_words_flag'].fillna(1)
# df_farewell = df_farewell.groupby(['stop_words_flag']).agg({'audio_dialog_fk_id': 'nunique'}).reset_index()
# df_farewell[df_farewell['farewell_phrase_flag'] == 0].head(20)
# df_farewell['stop_words'].unique()
df_farewell

,audio_dialog_fk_id,stop_words,stop_words_flag
0,001831ea-3b83-49de-89f8-4dc09f14b6a2,"""без разницы, в принципе, вы в параметрах увер...",1.0
1,001d827f-121e-4859-b0c0-ca4d65164d62,0,1.0
2,00378d6c-4458-46f7-baf1-0d2c57acbc06,"""але""""але""",1.0
3,0038e02c-60c8-4c4c-9b9d-9aeba288d41a,"""у нас нет""",1.0
4,003b5cec-6d0c-4cab-b1b1-346ef7c60feb,"""але""""але, нет такой информации, у нас нет""""ал...",1.0
...,...,...,...
3954,ffe2ebb9-bf6e-4273-b06c-6bb52fcea56b,"""информации нет, не в наличии""""не знаю""""я не м...",1.0
3955,ffe3d86d-2242-4c15-bf49-e9556a85d9f5,"""не знаю""""не знаю""""без разницы, в принципе, вы...",1.0
3956,ffe5dfc2-cbf3-488b-b608-f68dcea1dd9e,"""але""",1.0
3957,ffe8d4b2-b179-43ae-81a0-d8912fdc6bf8,"""в принципе""",1.0


In [229]:
import numpy as np
# df.groupby(['found_name']).agg({'audio_dialog_fk_id': 'nunique'})
df_name = df.copy()
df_name.loc[df_name['parasite_words'] == '""', 'parasite_words'] = np.nan
# df_name = df_name.groupby(['audio_dialog_fk_id']).agg({'parasite_words': 'sum'}).reset_index()
# df_name['found_name'].unique()
df_name.loc[(df_name['parasite_words'] == '') | (df_name['parasite_words'].isna()), 'parasite_words_flag'] = 0
df_name['parasite_words_flag'] = df_name['parasite_words_flag'].fillna(1)
df_name = df_name.groupby(['audio_dialog_fk_id']).agg({'parasite_words': 'nunique'}).reset_index()
df_name = df_name.groupby(['parasite_words']).agg({'audio_dialog_fk_id': 'nunique'}).reset_index()
df_name['share'] = df_name['audio_dialog_fk_id'] / df_name['audio_dialog_fk_id'].sum() * 100
# df_name[df_name['name_flag'] == 0]
df_name
# df_name['parasite_words'].unique()

,parasite_words,audio_dialog_fk_id,share
0,0,283,7.148270
1,1,595,15.029048
2,2,717,18.110634
3,3,613,15.483708
4,4,509,12.856782
5,5,405,10.229856
6,6,246,6.213690
7,7,181,4.571862
8,8,116,2.930033
9,9,107,2.702703


In [236]:
df_name = df.copy()
df_name.loc[df_name['parasite_words'] == '""', 'parasite_words'] = np.nan
# df_name = df_name.groupby(['parasite_words']).agg({'audio_dialog_fk_id': 'nunique'}).reset_index()
df_name = df_name.sort_values(['parasite_words'], ascending=False)
df_name

,dialog_criteria_id,dialog_row_fk_id,greeting_phrase,found_name,farewell_phrase,interjections,parasite_words,abbreviations,slang,inappropriate_phrases,diminutives,stop_words,swear_words,non_professional_phrases,order_offer,order_processing,order_resume,await_requests,id,audio_dialog_fk_id,row_num,row_text,speaker_id,start,end,id,file_name,status,duration,processing_time,created_at,updated_at
38967,de8598f9-8e69-4176-a8c4-7299746ac0dd,ee910b65-acea-4e13-bf85-a62ef6d29122,,,,"""""","""ясно""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""",None,ee910b65-acea-4e13-bf85-a62ef6d29122,81716b25-77bc-4afa-98be-1a9ef62109d0,36,"А, чуть-чуть потяжелее, да? Ясно. Да. Если рас...",SALES,240.0,250.8,81716b25-77bc-4afa-98be-1a9ef62109d0,in-07010101-89177519680-20250425-080645-174555...,PROCESSED,367.18,132.374755,2025-06-26 17:32:52.000000,2025-08-06 22:38:15.410543
6852,8b0f6e64-8d32-40ac-96ec-9886750ad508,be9e7897-632a-4078-aebf-9431d526c492,,,,"""""","""ясно""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""",None,be9e7897-632a-4078-aebf-9431d526c492,cde22132-c9eb-48a1-a98e-ebb2e1e8f5b6,39,"Да, ясно там. 5 магазинов в данном городе",SALES,321.8,326.0,cde22132-c9eb-48a1-a98e-ebb2e1e8f5b6,in-07010101-89853638133-20250403-145348-174368...,PROCESSED,382.70,126.167110,2025-06-26 19:57:10.000000,2025-08-07 01:01:07.380831
4103,8aa3e966-bc57-4176-bfff-72ab44135742,ab9cc0ed-d1be-441e-bcaa-1b540f172bfe,,,,"""""","""ясно""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""",None,ab9cc0ed-d1be-441e-bcaa-1b540f172bfe,d48b182f-fe88-4ab8-9010-ddfa5d943ce0,36,"Все ясно, хорошо. Цена не меняется, да? Нет, н...",SALES,685.5,691.9,d48b182f-fe88-4ab8-9010-ddfa5d943ce0,in-07010101-89109743999-20250408-130301-174410...,PROCESSED,702.44,121.801054,2025-06-26 18:57:05.000000,2025-08-07 00:04:49.659291
15268,63e8330a-10dc-49a6-b8c8-c591d998a4dc,a75b14e3-7275-4805-a1d0-49c351a19a9a,,,,"""""","""ясно""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""",None,a75b14e3-7275-4805-a1d0-49c351a19a9a,6afd0be0-28e8-44b6-8b27-b4163adcfd5c,103,"Нет, вроде все ясно.",SALES,1198.2,1200.3,6afd0be0-28e8-44b6-8b27-b4163adcfd5c,in-07010101-89901538566-20250423-133937-174540...,PROCESSED,1205.92,259.310933,2025-06-30 14:46:44.564865,2025-08-07 05:19:09.110939
4102,9e8f6be8-3577-4522-b2e1-0881c3394347,fe6266c8-e8a8-4f55-ba75-1bb3572e8477,,,,"""""","""ясно""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""",None,fe6266c8-e8a8-4f55-ba75-1bb3572e8477,d48b182f-fe88-4ab8-9010-ddfa5d943ce0,34,Ясно. Оплата при получении.,SALES,679.1,682.6,d48b182f-fe88-4ab8-9010-ddfa5d943ce0,in-07010101-89109743999-20250408-130301-174410...,PROCESSED,702.44,121.801054,2025-06-26 18:57:05.000000,2025-08-07 00:04:49.659291
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130567,e009c503-5bed-4e9c-991f-c11348f29c6d,42e63ceb-73fe-4911-b5ed-0244614f694b,,,,"""""",NaN,"""""","""""","""""","""""","""""","""""","""""","""""","""""","""""",None,42e63ceb-73fe-4911-b5ed-0244614f694b,c95dd2b4-a8fc-40d6-b937-9fb688dd6d67,22,Я могу посмотреть в Екатеринбурге сейчас инфор...,SALES,149.4,158.3,c95dd2b4-a8fc-40d6-b937-9fb688dd6d67,in-07010101-89090000723-20250430-202033-174603...,PROCESSED,1398.76,430.551619,2025-06-25 23:03:09.000000,2025-08-04 19:43:40.354620
130570,4f6489bb-463d-46fc-9d36-b64203404090,bf54da6a-5e89-479d-a9b3-2093d04ef9ff,,,,"""""",NaN,"""""","""""","""""","""""","""""","""""","""""","""""","""""","""""",None,bf54da6a-5e89-479d-a9b3-2093d04ef9ff,c95dd2b4-a8fc-40d6-b937-9fb688dd6d67,25,Екатеринбург Давайте я проверю.,SALES,179.6,181.0,c95dd2b4-a8fc-40d6-b937-9fb688dd6d67,in-07010101-89090000723-20250430-202033-174603...,PROCESSED,1398.76,430.551619,2025-06-25 23:03:09.000000,2025-08-04 19:43:40.354620
130571,86ad4da6-7a81-4eaf-9e99-85dba3584d5c,87e2593a-ccd2-41e9-aa80-1d4064ca5693,,,,"""""",NaN,"""""","""""","""""","""""

In [217]:
283/3676

0.07698585418933623

In [127]:
2634/(2634+5264)

0.3335021524436566

In [125]:
df = df.sort_values('row_num')
df[df['audio_dialog_fk_id'].astype(str) == '01ce075b-ca30-4d54-b1ff-e1ee8c9a1b7f']
# df_test = df[df['audio_dialog_fk_id'].isin((df_farewell[df_farewell['farewell_phrase_flag'] == 0]['audio_dialog_fk_id']))]
# df_test[df_test['row_num'] == 1]
# df_test[df_test['row_num'] == 1]
# df_test

,dialog_criteria_id,dialog_row_fk_id,greeting_phrase,found_name,farewell_phrase,interjections,parasite_words,abbreviations,slang,inappropriate_phrases,diminutives,stop_words,swear_words,non_professional_phrases,order_offer,order_processing,order_resume,await_requests,id,audio_dialog_fk_id,row_num,row_text,speaker_id,start,end,id,file_name,status,duration,processing_time,created_at,updated_at
54734,1f1412bb-cab9-4399-ba4b-c5c771c050b6,d7429c0f-c52b-4125-acd3-e12e0671ed80,,,,"""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""",None,d7429c0f-c52b-4125-acd3-e12e0671ed80,01ce075b-ca30-4d54-b1ff-e1ee8c9a1b7f,1,"Добрый день, Мирс Вячеслав. Хотел бы вам обращ...",SALES,0.0,3.4,01ce075b-ca30-4d54-b1ff-e1ee8c9a1b7f,in-07010101-89307037137-20250427-144736-174575...,PROCESSED,587.52,169.25606,2025-07-01 04:11:51.032600,2025-07-01 04:14:40.313391
181557,d3089074-59e1-477b-baee-982e783d049f,64e87cd6-3f8f-4971-9f56-eccccc2ac6bf,None,Владимир,None,"""""","""""","""""","""""","""""","""""","""""","""""",None,None,None,None,None,64e87cd6-3f8f-4971-9f56-eccccc2ac6bf,01ce075b-ca30-4d54-b1ff-e1ee8c9a1b7f,2,"Меня зовут Владимир, сейчас нахожусь вас на са...",CLIENT,3.4,8.7,01ce075b-ca30-4d54-b1ff-e1ee8c9a1b7f,in-07010101-89307037137-20250427-144736-174575...,PROCESSED,587.52,169.25606,2025-07-01 04:11:51.032600,2025-07-01 04:14:40.313391
181558,1a674b85-3464-414f-95b5-aae50ed55bc8,0078e26b-2f6b-41bd-8c3e-10e71d0d98d8,None,None,None,"""а.ш, эко""","""""","""""","""""","""""","""""","""""","""""",None,None,None,None,None,0078e26b-2f6b-41bd-8c3e-10e71d0d98d8,01ce075b-ca30-4d54-b1ff-e1ee8c9a1b7f,3,А.Ш.Некумхова Эко ВИНГ,CLIENT,10.1,12.3,01ce075b-ca30-4d54-b1ff-e1ee8c9a1b7f,in-07010101-89307037137-20250427-144736-174575...,PROCESSED,587.52,169.25606,2025-07-01 04:11:51.032600,2025-07-01 04:14:40.313391
181559,24b024fe-263e-4c19-b491-b3ae25d2ccfd,862947d1-d152-496c-833b-e5269b02cb28,None,None,None,"""ес-""","""""","""""","""""","""""","""""","""""","""""",None,None,None,None,None,862947d1-d152-496c-833b-e5269b02cb28,01ce075b-ca30-4d54-b1ff-e1ee8c9a1b7f,4,ЕС-31 175 или 190.,CLIENT,13.8,16.1,01ce075b-ca30-4d54-b1ff-e1ee8c9a1b7f,in-07010101-89307037137-20250427-144736-174575...,PROCESSED,587.52,169.25606,2025-07-01 04:11:51.032600,2025-07-01 04:14:40.313391
181560,74b7a2d9-bde8-455a-b90c-fed27cae898d,38d4ef9d-2f97-450f-ad75-11477ecdd4b8,None,None,None,"""""","""""","""""","""""","""""","""""","""радиус""","""""",None,None,None,None,None,38d4ef9d-2f97-450f-ad75-11477ecdd4b8,01ce075b-ca30-4d54-b1ff-e1ee8c9a1b7f,5,Радиус 15 вершины.,CLIENT,19.5,21.4,01ce075b-ca30-4d54-b1ff-e1ee8c9a1b7f,in-07010101-89307037137-20250427-144736-174575...,PROCESSED,587.52,169.25606,2025-07-01 04:11:51.032600,2025-07-01 04:14:40.313391
181561,f21e07e1-fe6f-4171-8577-0a1b2914c957,54aedce6-e51e-4275-88e5-0d476a164594,,,,"""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""",None,54aedce6-e51e-4275-88e5-0d476a164594,01ce075b-ca30-4d54-b1ff-e1ee8c9a1b7f,6,"Ну, все. Параметры еще 1.85.",SALES,21.4,24.8,01ce075b-ca30-4d54-b1ff-e1ee8c9a1b7f,in-07010101-89307037137-20250427-144736-174575...,PROCESSED,587.52,169.25606,2025-07-01 04:11:51.032600,2025-07-01 04:14:40.313391
181562,958d6b3f-802a-4bce-a4f0-4714419f03af,d93ba33e-81b3-4d9c-95d0-2bf60a27f815,None,None,None,"""""","""""","""""","""""","""""","""""","""радиус""","""""",None,None,None,None,None,d93ba33e-81b3-4d9c-95d0-2bf60a27f815,01ce075b-ca30-4d54-b1ff-e1ee8c9a1b7f,7,"На 95 верно? На 60. Можно 185 на 65, главный р...",CLIENT,24.8,33.5,01ce075b-ca30-4d54-b1ff-e1ee8c9a1b7f,in-07010101-89307037137-20250427-144736-174575...,PROCESSED,587.52,169.25606,2025-07-01 04:11:51.032600,2025-07-01 04:14:40.313391
54752,2809bd7b-7e63-483c-ae56-c668f4fbaee8,6be67b0e-8b6f-4191-948a-60f96ad3da39,,,,"""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""",None,6be67b0e-8b6f-4191-948a-60f96ad3da39,01ce075b-ca30-4d54-b1ff-e1ee8c9a1b7f,8,"Да, я вас понял. Отставьте, по

In [123]:
df['row_text'][168541]

'Да, всё российское, но там города разные. Понял. '

In [16]:
df.columns

Index(['dialog_criteria_id', 'dialog_row_fk_id', 'greeting_phrase',
       'found_name', 'farewell_phrase', 'interjections', 'parasite_words',
       'abbreviations', 'slang', 'inappropriate_phrases', 'diminutives',
       'stop_words', 'swear_words', 'non_professional_phrases', 'order_offer',
       'order_processing', 'order_resume', 'await_requests', 'id',
       'audio_dialog_fk_id', 'row_num', 'row_text', 'speaker_id', 'start',
       'end'],
      dtype='object')

In [9]:
df = pd.read_sql(f"""
SELECT *
from dialog_criterias t1
inner join dialog_rows t2 on t1.dialog_row_fk_id = t2.id
-- left join audio_dialog t3 on t2.audio_dialog_fk_id = t3.id
where found_name is not null and swear_words != '""'
""", engine)
df

,dialog_criteria_id,dialog_row_fk_id,greeting_phrase,found_name,farewell_phrase,interjections,parasite_words,abbreviations,slang,inappropriate_phrases,diminutives,stop_words,swear_words,id,audio_dialog_fk_id,row_num,row_text,speaker_id,start,end
0,10ab6063-fd00-4cf3-ad67-295a39a618f1,f8ed450b-968c-4394-83cf-6a917607eda4,None,Раиль,None,а,да,,,,,,,f8ed450b-968c-4394-83cf-6a917607eda4,c95dd2b4-a8fc-40d6-b937-9fb688dd6d67,126,"Раниль, да, сказали вас зовут? Раиль. А, Раил...",SALES,1052.1,1054.9
1,95f7152e-65c6-4e90-9991-da15c4c18985,4436e47e-ac8d-479b-9213-66361cfe2e99,None,Пирелли,None,"""\u0430""","""\u0432\u043e\u043e\u0431\u0449\u0435, \u043f\...","""""","""""","""""","""\u0442\u0443\u0442""","""""","""\u043e\u0434\u0438\u043d""",4436e47e-ac8d-479b-9213-66361cfe2e99,6c1d42ca-7f55-4772-a70b-53097e132734,55,"Я вам подскажу, по ССМ, вообще они считаются к...",SALES,597.1,611.1
2,76fdd0f1-52a5-47ec-a6f2-0ddb5d8a4954,410de36c-2e83-407c-b01f-d6a62750e235,добрый вечер,Алиса Кеммка,до свидания,,,,,,,,,410de36c-2e83-407c-b01f-d6a62750e235,516cc52e-8bf9-45b9-8ddc-b93b5d7da9b2,98,"До свидания. Добрый вечер, меня зовут Алиса Ке...",SALES,905.6,912.7
3,ba674dc6-20b5-4815-a485-01a2c1b01660,4e37353d-b883-4f39-bae3-a3e9dc8b979a,None,Восковой,None,это,это,,,,,,,4e37353d-b883-4f39-bae3-a3e9dc8b979a,d96ca131-4e21-461a-92b6-e2ee6c7b423e,71,"На любом, на Римского, Корского или на Восково...",CLIENT,574.5,581.1
4,2d71a22e-34cf-432d-b6eb-f034f4b062a9,000d545d-1727-4bb4-9b1e-0026e4a4a3f3,None,Угу,None,"а, это","да, угу, это",,,,,"не могу, не могу помочь",,000d545d-1727-4bb4-9b1e-0026e4a4a3f3,645edaa5-c1f2-4ac3-abdf-ffb748d1bfea,61,"240 на 45 да это заднее колесо Да, всё верно....",CLIENT,471.1,477.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
273,ff7c74ec-5f3c-4311-831b-5e01ed35d37d,be8112a6-01bd-4223-b7ce-78582744ccae,добрый день,Елена,None,"""\u0435\u0433\u043e""","""\u0432\u043e\u0442""","""""","""""","""""","""""","""""","""\u043e\u0434\u0438\u043d""",be8112a6-01bd-4223-b7ce-78582744ccae,b7c96fbe-a0a9-4e4a-9aac-f868a976bd9d,1,"Добрый день, меня зовут Елена, как могут к вам...",SALES,0.0,6.4
274,298244a6-caf1-45f0-b79a-7df8f8877201,e1527c5c-12b4-40a1-9780-64f06238fdd9,добрый день,Анастасия,None,"""""","""""","""""","""""","""""","""""","""""","""\u0433\u0430\u0434\u0438\u0441""",e1527c5c-12b4-40a1-9780-64f06238fdd9,80b0c716-5704-4ec4-8bdb-201d7d7b4493,1,"Добрый день, меня зовут Анастасия. Как я могу ...",SALES,0.0,6.5
275,44f050ac-a3f3-4913-8ff5-41d7c8e86754,03e62b2e-257d-45c0-a86f-7b05702f155b,здравствуйте,Вопрос,None,"""""","""""","""""","""""","""""","""""","""""","""\u043e\u0434\u0438\u043d""",03e62b2e-257d-45c0-a86f-7b05702f155b,697d88e2-9181-4aa2-b358-5b07ebe53e90,1,здравствуйте меня зовут артур как я могу к вам...,SALES,0.0,10.4
276,af2288cd-011a-4e0b-a90f-426f64fc4b17,f939fee7-9ed8-40d4-aa0e-9f18550df82e,добрый день,Екатерина,None,"""""","""""","""""","""""","""""","""""","""""","""\u0431\u043b\u044f\u0434\u044c""",f939fee7-9ed8-40d4-aa0e-9f18550df82e,0fe08002-33f3-426b-90f9-7ef7c89b25a1,4,"Я паю. Блядь. Добрый день, я зовут Екатерина. ...",SALES,136.5,173.0


In [80]:
df.groupby('has_stopwords').agg({'audio_dialog_fk_id': 'nunique'})
# df[df['has_swear_words'] == 1]

,audio_dialog_fk_id
has_stopwords,
0,13
1,153


In [37]:
df = pd.read_sql(f"""
SELECT file_name, row_text, swear_words, stop_words, t1.*
-- t2.audio_dialog_fk_id, t1.*, speaker_id
from dialog_rows t2
left join dialog_criterias t1 on t1.dialog_row_fk_id = t2.id
left join audio_dialogs t3 on t2.audio_dialog_fk_id = t3.id
where audio_dialog_fk_id = 'fb18a776-3b4b-44ec-962a-7cdfea0bb104'
and speaker_id = 'SALES'
-- group by 1
and swear_words != '""'
order by row_num
""", engine)
df

,file_name,row_text,swear_words,stop_words,dialog_criteria_id,dialog_row_fk_id,greeting_phrase,found_name,farewell_phrase,interjections,parasite_words,abbreviations,slang,inappropriate_phrases,diminutives,stop_words,swear_words,non_professional_phrases,order_offer,order_processing,order_resume,await_requests
0,in-07010101-83412933199-20250402-111618-174358...,"Да, я вам сейчас озвучу, давайте я вам по айко...","""хуйно""","""у нас нет, я же говорим""",2dcbb144-761d-4088-8b8a-f7df4d630635,91e80f92-f769-48df-96b6-636ac829e713,None,Нордман,None,"""а, это""","""то есть, это""","""""","""""","""""","""""","""у нас нет, я же говорим""","""хуйно""",None,None,None,None,""""""


In [88]:
df['row_text'][6]

'И как раз с пиццом уже подкашьте. Ну, нормы вполне могут быть еще. Хорошо. '